In [69]:
import os
os.environ["CEA_USE_SITE_PACKAGES"] = '1'
from CEA_Wrap import Fuel, Oxidizer, RocketProblem

from utils import bar_to_psi
import numpy as np

## 0. Constants

Natural

In [70]:
G0 = 9.81                   # m/s^2

Chemical

In [71]:
ipa = Fuel("C3H8O,2propanol", temp=293.15)  # Liquid IPA at 20 C
n2o = Oxidizer("N2O", temp=283.15)        # Nitrous oxide at 10 C

In [72]:
COMBUSTION_OF_RATIO = 3           # Oxidizer to fuel ratio by mass

Mechanical

In [73]:
THRUST = 2000               # N

In [74]:
CHAMBER_PRESSURE_PA = 2.0e6 # Pa (20 bar)
CHAMBER_PRESSURE_BAR = CHAMBER_PRESSURE_PA / 1e5   # Chamber pressure in bar
EXIT_PRESSURE_BAR = 1.0     # Sea level a minute

In [75]:
pressure_ratio = CHAMBER_PRESSURE_BAR / EXIT_PRESSURE_BAR
pressure_ratio

20.0

## 1. Consider film cooling is part of chemistry
It won't burn because we're short of oxidiser anyway 

In [76]:
COMBUSTION_FUEL_MASS_FLOW = 1.0 # Normalized fuel mass flow rate, will cancel out

OXIDISER_MASS_FLOW = COMBUSTION_OF_RATIO * COMBUSTION_FUEL_MASS_FLOW

TOTAL_FUEL_MASS_FLOW = COMBUSTION_FUEL_MASS_FLOW * 1.1 # Add 10% for film cooling

NET_OF_RATIO = OXIDISER_MASS_FLOW / TOTAL_FUEL_MASS_FLOW

NET_OF_RATIO

2.727272727272727

NASA CEA analysis

In [77]:
problem = RocketProblem(
    pressure=bar_to_psi(CHAMBER_PRESSURE_BAR),
    materials=[ipa, n2o],
    o_f=NET_OF_RATIO,
    pip=pressure_ratio,
)

In [78]:
results = problem.run()

Key results

In [79]:
print(f"Chamber Temperature (K):            {results.c_t:.1f}")
print(f"Chamber Pressure (bar):             {results.c_p:.1f}")
print(f"Exit Temperature (K):               {results.t:.1f}")
print(f"Exit Pressure (bar):                {results.p:.1f}")
print(f"Theoretical Specific Impulse (s):   {results.isp:.1f}")
print(f"Characteristic Velocity (m/s):      {results.cstar:.1f}")
print(f"Thrust Coefficient:                 {results.cf:.3f}")
print(f"Specific Heat Ratio (gamma):        {results.gamma:.2f}")
print(f"Area Ratio (Expansion ratio):       {results.ae:.4f}")
print(f"Final Mach Number:                  {results.mach:.4f}")


Chamber Temperature (K):            2370.1
Chamber Pressure (bar):             20.0
Exit Temperature (K):               1245.9
Exit Pressure (bar):                1.0
Theoretical Specific Impulse (s):   208.6
Characteristic Velocity (m/s):      1471.5
Thrust Coefficient:                 1.391
Specific Heat Ratio (gamma):        1.28
Area Ratio (Expansion ratio):       3.2940
Final Mach Number:                  2.5678


In [80]:
mass_flow_rate = THRUST / (results.isp * G0)
print(f"Mass Flow Rate (kg/s):              {mass_flow_rate:.4f}")

Mass Flow Rate (kg/s):              0.9774


In [81]:
throat_area = mass_flow_rate * results.cstar / CHAMBER_PRESSURE_PA # m^2
print(f"Throat Area (m^2):                  {throat_area:.6f}")

Throat Area (m^2):                  0.000719


In [82]:
throat_radius = (throat_area / np.pi) ** 0.5  # m
print(f"Throat Radius (m):                  {throat_radius:.4f}")
print(f"Throat Diameter (m):                {2 * throat_radius:.4f}")
print(f"Throat Diameter (mm):               {2 * throat_radius * 1e3:.2f}")

Throat Radius (m):                  0.0151
Throat Diameter (m):                0.0303
Throat Diameter (mm):               30.26


## 2. Ignore film cooling

In [83]:
problem = RocketProblem(
    pressure=bar_to_psi(CHAMBER_PRESSURE_BAR),
    materials=[ipa, n2o],
    o_f=COMBUSTION_OF_RATIO,
    pip=pressure_ratio,
)

In [84]:
results = problem.run()

In [85]:
print(f"Chamber Temperature (K):            {results.c_t:.1f}")
print(f"Chamber Pressure (bar):             {results.c_p:.1f}")
print(f"Exit Temperature (K):               {results.t:.1f}")
print(f"Exit Pressure (bar):                {results.p:.1f}")
print(f"Theoretical Specific Impulse (s):   {results.isp:.1f}")
print(f"Characteristic Velocity (m/s):      {results.cstar:.1f}")
print(f"Thrust Coefficient:                 {results.cf:.3f}")
print(f"Specific Heat Ratio (gamma):        {results.gamma:.2f}")
print(f"Area Ratio (Expansion ratio):       {results.ae:.4f}")
print(f"Final Mach Number:                  {results.mach:.4f}")

Chamber Temperature (K):            2550.2
Chamber Pressure (bar):             20.0
Exit Temperature (K):               1366.9
Exit Pressure (bar):                1.0
Theoretical Specific Impulse (s):   213.6
Characteristic Velocity (m/s):      1505.4
Thrust Coefficient:                 1.392
Specific Heat Ratio (gamma):        1.27
Area Ratio (Expansion ratio):       3.3263
Final Mach Number:                  2.5624


In [86]:
combustion_mass_flow_rate = THRUST / (results.isp * G0)
print(f"Combustion Mass Flow Rate (kg/s):     {combustion_mass_flow_rate:.4f}")

Combustion Mass Flow Rate (kg/s):     0.9545


In [87]:
throat_area = mass_flow_rate * results.cstar / CHAMBER_PRESSURE_PA # m^2
print(f"Throat Area (m^2):                  {throat_area:.6f}")

Throat Area (m^2):                  0.000736


In [88]:
throat_radius = (throat_area / np.pi) ** 0.5  # m
print(f"Throat Radius (m):                  {throat_radius:.4f}")
print(f"Throat Diameter (m):                {2 * throat_radius:.4f}")
print(f"Throat Diameter (mm):               {2 * throat_radius * 1e3:.2f}")

Throat Radius (m):                  0.0153
Throat Diameter (m):                0.0306
Throat Diameter (mm):               30.61


## 3. Tack on film cooling outside combustion

In [89]:
combustion_fuel_flow_rate = combustion_mass_flow_rate / (1 + COMBUSTION_OF_RATIO)
print(f"Combustion Fuel Mass Flow Rate (kg/s): {combustion_fuel_flow_rate:.4f}")

Combustion Fuel Mass Flow Rate (kg/s): 0.2386


In [90]:
film_cooling_fuel_flow_rate = combustion_fuel_flow_rate * 0.1
print(f"Film Cooling Fuel Mass Flow Rate (kg/s): {film_cooling_fuel_flow_rate:.4f}")

Film Cooling Fuel Mass Flow Rate (kg/s): 0.0239


In [91]:
total_mass_flow_rate = combustion_mass_flow_rate + film_cooling_fuel_flow_rate
print(f"Total Mass Flow Rate (kg/s):           {total_mass_flow_rate:.4f}")

Total Mass Flow Rate (kg/s):           0.9784


In [92]:
throat_area = total_mass_flow_rate * results.cstar / CHAMBER_PRESSURE_PA # m^2
print(f"Throat Area (m^2):                  {throat_area:.6f}")

Throat Area (m^2):                  0.000736


In [93]:
throat_radius = (throat_area / np.pi) ** 0.5  # m
print(f"Throat Radius (m):                  {throat_radius:.4f}")
print(f"Throat Diameter (m):                {2 * throat_radius:.4f}")
print(f"Throat Diameter (mm):               {2 * throat_radius * 1e3:.2f}")

Throat Radius (m):                  0.0153
Throat Diameter (m):                0.0306
Throat Diameter (mm):               30.62
